In [1]:
import os
import sys

import allel
import dask.array as da
import numcodecs
import numpy as np
import requests
import zarr

In [2]:
url_toy_vcf = "https://raw.githubusercontent.com/samtools/hts-specs/master/examples/vcf/simple.vcf"

In [3]:
outdir = "data/"

In [4]:
os.makedirs(outdir, exist_ok=True)
fname_simple_zarr = os.path.join(outdir, "toy.zarr")

In [5]:
response = requests.get(url_toy_vcf, stream=True)
response.raw.decode_content = True

In [6]:
# https://scikit-allel.readthedocs.io/en/latest/io.html#allel.vcf_to_zarr
# http://alimanfoo.github.io/2016/09/21/genotype-compression-benchmark.html
allel.vcf_to_zarr(response.raw, 
                  fname_simple_zarr, 
                  fields="*", 
                  compressor=numcodecs.Blosc(cname='zstd', clevel=1, shuffle=False), 
                  chunk_length=2,
                  chunk_width=1,
                  log=sys.stdout, 
                  overwrite=True)

[vcf_to_zarr] 2 rows in 0.00s; chunk in 0.00s (16513 rows/s); 20 :17330
[vcf_to_zarr] 4 rows in 0.10s; chunk in 0.10s (20 rows/s); 20 :1230237
[vcf_to_zarr] 5 rows in 0.15s; chunk in 0.05s (20 rows/s); 20 :1234567
[vcf_to_zarr] all done (26 rows/s)


In [7]:
callset = zarr.open_group(fname_simple_zarr, mode="r")

In [8]:
callset

<zarr.hierarchy.Group '/' read-only>

In [10]:
callset["samples"]

<zarr.core.Array '/samples' (3,) object read-only>

In [11]:
callset["samples"][:]

array(['NA00001', 'NA00002', 'NA00003'], dtype=object)

In [12]:
callset["calldata/GT"]

<zarr.core.Array '/calldata/GT' (5, 3, 2) int8 read-only>

In [13]:
gt = callset["calldata/GT"]

In [14]:
gt[0]

array([[0, 0],
       [1, 0],
       [1, 1]], dtype=int8)

In [15]:
allel.GenotypeArray(callset['calldata/GT'])

,0,1,2
0,0/0,1/0,1/1
1,0/0,0/1,0/0
2,1/2,2/1,2/2
3,0/0,0/0,0/0
4,0/1,0/2,1/1


In [16]:
simple_da = da.from_zarr(os.path.join(fname_simple_zarr, "calldata/GT/"))

In [17]:
simple_da

dask.array<from-zarr, shape=(5, 3, 2), dtype=int8, chunksize=(2, 1, 2), chunktype=numpy.ndarray>

In [18]:
simple_da[0].compute()

array([[0, 0],
       [1, 0],
       [1, 1]], dtype=int8)

In [19]:
allele_dosage = da.apply_along_axis(np.sum, -1, simple_da)

In [20]:
allele_dosage

dask.array<sum-along-axis, shape=(5, 3), dtype=int64, chunksize=(2, 1), chunktype=numpy.ndarray>

In [21]:
allele_dosage[:].compute()

array([[0, 1, 2],
       [0, 1, 0],
       [3, 3, 4],
       [0, 0, 0],
       [1, 2, 2]])